In [13]:
import csv
import json
import requests

In [14]:
def analyze_entity_sentiment(articles):
    api_endpoint = "https://language.googleapis.com/v1/documents:analyzeEntitySentiment"
    api_key = ""  # Replace with your actual API key
    
    entity_sentiment_results = []  # List to store entity sentiment analysis results
    
    for idx, article in enumerate(articles, start=1):
        text_content = article["Text Content"]
        article_title = article["Article Title"]
        
        # Create the request payload for entity sentiment analysis
        data = {
            "document": {
                "type": "PLAIN_TEXT",
                "content": text_content
            },
            "encodingType": "UTF8",
        }
        
        params = {
            "key": api_key
        }
        
        headers = {
            "Content-Type": "application/json"
        }
        
        # Send the HTTP POST request for entity sentiment analysis
        response = requests.post(api_endpoint, params=params, headers=headers, data=json.dumps(data))
        
        if response.status_code == 200:
            entity_sentiment_data = response.json()
            # Get the entity sentiment analysis results
            entity_sentiments = entity_sentiment_data.get("entities", [])
            for entity_sentiment in entity_sentiments:
                entity_sentiment_result = {
                    "Article Number": idx,
                    "Article Title": article_title,
                    "Entity Name": entity_sentiment["name"],
                    "Entity Type": entity_sentiment["type"],
                    "Entity Salience": entity_sentiment["salience"],
                    "Entity Sentiment Score": entity_sentiment["sentiment"]["score"],
                    "Entity Sentiment Magnitude": entity_sentiment["sentiment"]["magnitude"]
                }
                entity_sentiment_results.append(entity_sentiment_result)
        
    # Analyze mentions for the entity
                # mentions = entity_sentiment.get("mentions", [])
                # for mention in mentions:
                #     mention_result = {
                #         "Entity Name": entity_sentiment["name"],
                #         "Mention Content": mention["text"]["content"],
                #         "Mention Type": mention["type"],
                #     }
                #     entity_sentiment_results.append(mention_result)
        else:
            print(f"Error for Article {idx}: {response.status_code} - {response.text}")
            continue
    
    
    # Save the entity sentiment analysis results to a CSV file
    csv_file = "entity_sentiment_results.csv"
    csv_headers = ["Article Number", "Article Title", "Entity Name", "Entity Type", "Entity Salience", "Entity Sentiment Score", "Entity Sentiment Magnitude"]
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=csv_headers)
        writer.writeheader()
        writer.writerows(entity_sentiment_results)
    
    print(f"Entity sentiment analysis results saved to {csv_file}")

if __name__ == "__main__":
    # Read articles and their text content from a CSV file
    articles = []
    with open('articles.csv', mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            articles.append({"Article Title": row['Article Title'], "Text Content": row['Text Content']})
    
    analyze_entity_sentiment(articles)


Entity sentiment analysis results saved to entity_sentiment_results.csv
